In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.metrics import log_loss,roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer


from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.utils import shuffle
import pickle
from sklearn.model_selection import KFold, StratifiedKFold
import numpy as np
import gc
import time

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load Data

In [2]:
train = pd.read_pickle(FILE.train_ori.value)
test = pd.read_pickle(FILE.test_ori.value)
train_doc = pd.read_pickle('../../data/fe_doc/train_doc.pkl')
test_doc = pd.read_pickle('../../data/fe_doc/test_doc.pkl')


holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))
train_index = pickle.load(open(FILE.train_index.value,'rb'))
train_cv = train_doc.loc[train_index].copy()
holdout = train_doc.loc[holdout_index].copy()

test_matrix = pickle.load(open(FILE.countVectorize_format.value.format('test'),'rb'))

In [3]:
# df = pd.DataFrame()
# for fold in range(5):
#     index_name = FILE.Vectorize_index_format.value.format('val_fold_{}'.format(fold))
#     index = pickle.load(open(index_name,'rb'))
#     label_name = FILE.Vectorize_label_format.value.format('val_fold_{}'.format(fold))
#     label = pickle.load(open(label_name,'rb'))
#     dfc = pd.DataFrame({'idx':index,'label':label})
#     df = pd.concat([df,dfc])
# label_ho = pickle.load(open(FILE.Vectorize_label_format.value.format('holdout'),'rb'))
# dfc = pd.DataFrame({'idx':holdout_index,'label':label_ho})
# df = pd.concat([df,dfc])
# df = df.sort_values('idx')

# Define NN model

In [4]:
param_list = []
for layer1 in [4096,5120,6144]:
    for layer2 in [None,1024,2048]:
        for layer3 in [None]:
            if layer2 is None and layer3 is not None:
                continue
            if layer2 is not None and layer3 is not None:
                if layer2 < layer3:
                    continue
            param = {'input_shape':test_matrix.shape[1],
                     'layer1':layer1,
                     'layer2':layer2,
                     'layer3':layer3}
            param_list.append(param)
print(len(param_list))

train_batch = 2048
test_batch = 20000




def get_nn_model(param):
    layer1 = param.get('layer1')
    layer2 = param.get('layer2')
    layer3 = param.get('layer3')
    input_shape = param.get('input_shape')
    
    sequence_input = Input(shape=(input_shape, ))
    x = Dense(layer1, activation='relu')(sequence_input)
    x = Dropout(0.5)(x)
    
    if layer2 is not None:
        x = Dense(layer2, activation='relu')(x)
        x = Dropout(0.5)(x)
    
    if layer2 is not None and layer3 is not None:
        x = Dense(layer3, activation='relu')(x)
        x = Dropout(0.2)(x)
        
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
    return model

def train_each_epoch(x,y,batch_size,model):
    x,y = shuffle(x,y)
    model.fit(x, y, 
              batch_size=batch_size, 
              epochs=1,
              verbose=1,
              )
    return model

def get_file_name(param,vectorize='countVectorize'):
    layer1 = param.get('layer1')
    layer2 = param.get('layer2')
    layer3 = param.get('layer3')
    return '{}_layer1_{}_layer2_{}_layer3_{}.pkl'.format(vectorize,layer1,layer2,layer3)

def save_report(params,report_path):
    """
    params. dataframe report params.
    """
    try:
        saved_report = pd.read_csv(report_path)
    except:
        saved_report = pd.DataFrame()
    new_report = pd.DataFrame({'fileName':params['file_Name'],
                               'cv_mean':params['cv_mean'],
                               'holdout':params['holdout']})
    saved_report = pd.concat([saved_report,new_report]).reset_index(drop=True)
    saved_report.to_csv(report_path)
    print('saved report to path {}'.format(report_path))
    
def save_preds(file_name,preds_params,path='../../data/nn/{}/{}'):
    """
    param: nn layer info.
    preds_params:key should be train/test/holdout and values should be coresponding predict dataframe
    """
    for key in preds_params:
        saving_path = path.format(key,file_name)
        preds_params[key].to_pickle(saving_path)
        print('saving preds {} done!'.format(key))
        
def train_each_fold(param,fold,mode='countVectorize',tolerance=2):
    if mode != 'tfidf':
        train_name = FILE.countVectorize_format.value.format('train_fold_{}'.format(fold))
        val_name = FILE.countVectorize_format.value.format('val_fold_{}'.format(fold))
        holdout_name = FILE.countVectorize_format.value.format('holdout')
        test_name = FILE.countVectorize_format.value.format('test')
        
    else:
        train_name = FILE.tfidfVectorize_format.value.format('train_fold_{}'.format(fold))
        val_name = FILE.tfidfVectorize_format.value.format('val_fold_{}'.format(fold))
        holdout_name = FILE.tfidfVectorize_format.value.format('holdout')
        test_name = FILE.tfidfVectorize_format.value.format('test')
        
    y_train_name = FILE.Vectorize_label_format.value.format('train_fold_{}'.format(fold))
    y_val_name = FILE.Vectorize_label_format.value.format('val_fold_{}'.format(fold))
    y_holdout_name = FILE.Vectorize_label_format.value.format('holdout')
        
    x_train = pickle.load(open(train_name,'rb'))
    x_val = pickle.load(open(val_name,'rb'))
    x_holdout = pickle.load(open(holdout_name,'rb'))
    x_test = pickle.load(open(test_name,'rb'))
    y_train =   pickle.load(open(y_train_name,'rb'))
    y_val =   pickle.load(open(y_val_name,'rb'))
    y_holdout =   pickle.load(open(y_holdout_name,'rb'))
    tol = 0
    model = get_nn_model(param)
    best_loss = None
    while True:
        model = train_each_epoch(x_train,y_train,train_batch,model)
        val_pred = model.predict(x_val,test_batch,verbose=1)
        score = log_loss(y_val,val_pred)
        print('current validation loss: {}'.format(score))
        
        if best_loss is None:
            tol = 0
            best_loss = score
            holdout_preds = model.predict(x_holdout,test_batch,verbose=1)
            holdout_score = log_loss(y_holdout,holdout_preds)
            print('holdout loss is: {}'.format(holdout_score))
            test_preds = model.predict(x_test,test_batch,verbose=1)
            continue
        if score < best_loss:
            tol = 0
            best_loss = score
            holdout_preds = model.predict(x_holdout,test_batch,verbose=1)
            holdout_score = log_loss(y_holdout,holdout_preds)
            print('holdout loss is: {}'.format(holdout_score))
            test_preds = model.predict(x_test,test_batch,verbose=1)
        else:
            tol += 1
            if tol == tolerance:
                break
        print('best validation loss: {}'.format(best_loss))
    return best_loss,val_pred,holdout_preds,test_preds

def train_5_fold(train,test,param,vectorize='countVectorize',report_path='../../data/nn/report.csv'):
    train_oof = train[['instance_id']].copy()
    test_oof = test[['instance_id']].copy()
    train_oof['predicted_score'] = np.nan
    file_name = get_file_name(param,vectorize)
    cv_list = []
    holdout_list = []
    test_list = []
    try:
        saved_report = pd.read_csv(report_path)
        if file_name in saved_report['fileName'].values:
            return None
    except:
        print('no saved report found. create a new one')
        saved_report = pd.DataFrame()
    for fold in range(7):
        print('start fold {}...'.format(fold))
        score,val_pred,holdout_preds,test_preds =  train_each_fold(param,fold,mode=vectorize,tolerance=2)
        cv_list.append(score)
        holdout_list.append(holdout_preds)
        test_list.append(test_preds)
        index_name = FILE.Vectorize_index_format.value.format('val_fold_{}'.format(fold))
        val_index = pickle.load(open(index_name,'rb'))
        train_oof.loc[val_index,'predicted_score'] = val_pred
        gc.collect()
        time.sleep(5)
        
    
    holdout = np.mean(holdout_list,axis=0)
    test = np.mean(test_list,axis=0)
    cv_mean = np.mean(cv_list)
    y_holdout_name = FILE.Vectorize_label_format.value.format('holdout')
    y_holdout =   pickle.load(open(y_holdout_name,'rb'))
    holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))
    train_oof.loc[holdout_index,'predicted_score'] = holdout
    test_oof['predicted_score'] = test
    holdout_df =  train_oof.loc[holdout_index].copy()
    
    holdout_score = log_loss(y_holdout,holdout)
    print('cv mean: {}, overall holdout: {}'.format(cv_mean,holdout_score))
    
    
    new_report = pd.DataFrame({'fileName':[file_name],
                               'cv_mean':[cv_mean],
                               'holdout':[holdout_score]})
   
    saved_report = pd.concat([saved_report,new_report],sort=False).reset_index(drop=True)
    saved_report.to_csv(report_path,index=False)
    print('saved report to path {}'.format(report_path))
    
    preds_param = {'train':train_oof,'test':test_oof,'holdout':holdout_df}
    
    save_preds(file_name,preds_param)
    
    
    
    

9


# Train

In [5]:
for param in param_list:
    print(param)
    train_5_fold(train,test,param,vectorize='countVectorize',report_path='../../data/nn/report.csv')
    gc.collect()

{'layer1': 4096, 'layer2': None, 'layer3': None, 'input_shape': 25422}
start fold 0...
Epoch 1/1
135491/135491 [==============================] - 21s 152us/step
current validation loss: 0.4114878781143221
40024/40024 [==============================] - 6s 154us/step
holdout loss is: 0.4276503681903065
40024/40024 [==============================] - 6s 150us/step
Epoch 1/1
135491/135491 [==============================] - 21s 152us/step
current validation loss: 0.4117469063793765
best validation loss: 0.4114878781143221
Epoch 1/1
135491/135491 [==============================] - 21s 154us/step
current validation loss: 0.411922317119572
start fold 1...
Epoch 1/1
142734/142734 [==============================] - 21s 150us/step
current validation loss: 0.4216130411413319
40024/40024 [==============================] - 6s 156us/step
holdout loss is: 0.42865379437322065
40024/40024 [==============================] - 6s 154us/step
Epoch 1/1
142734/142734 [==============================] - 22s 152us

ResourceExhaustedError: OOM when allocating tensor with shape[25422,4096]
	 [[Node: training_4/Adam/mul_2 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_4/Adam/sub_2, training_4/Adam/gradients/dense_9/MatMul_grad/MatMul_1)]]

Caused by op 'training_4/Adam/mul_2', defined at:
  File "/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/kai/anac

In [ ]:
report_path='../../data/nn/report.csv'
saved_report = pd.read_csv(report_path)
saved_report1 = pd.read_csv(report_path)

In [ ]:
pd.concat([saved_report,saved_report1]).reset_index(drop=True)

In [ ]:
new_report = pd.DataFrame({'fileName':['123.csv'],
                               'cv_mean':[0.23232],
                               'holdout':[0.12121]})

In [ ]:
pd.concat([saved_report,new_report]).reset_index(drop=True)